<a href="https://colab.research.google.com/github/iqshansyarifudin/DasarAlgoritmaDanPemrograman-BisDig/blob/main/Miqshan_syarifudin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import csv
import json
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Optional
import hashlib
import logging
from abc import ABC, abstractmethod

# Konfigurasi logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('inventory_system.log'),
        logging.StreamHandler()
    ]
)

## Model Data ##
@dataclass
class Product:
    id: str
    name: str
    category: str
    price: float
    quantity: int
    min_stock: int = 5

@dataclass
class Transaction:
    id: str
    product_id: str
    type: str  # 'in' or 'out'
    quantity: int
    timestamp: str
    notes: Optional[str] = None

## Strategy Pattern untuk Ekspor Data ##
class ExportStrategy(ABC):
    @abstractmethod
    def export(self, data: List[Dict]) -> str:
        pass

class CSVExportStrategy(ExportStrategy):
    def export(self, data: List[Dict]) -> str:
        if not data:
            return ""

        filename = f"export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        with open(filename, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)
        return filename

class JSONExportStrategy(ExportStrategy):
    def export(self, data: List[Dict]) -> str:
        filename = f"export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        return filename

## Database Manager ##
class DatabaseManager:
    def __init__(self, db_name: str = 'inventory.db'):
        self.db_name = db_name
        self._init_db()

    def _init_db(self):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()

            # Tabel produk
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS products (
                    id TEXT PRIMARY KEY,
                    name TEXT NOT NULL,
                    category TEXT NOT NULL,
                    price REAL NOT NULL,
                    quantity INTEGER NOT NULL,
                    min_stock INTEGER DEFAULT 5
                )
            ''')

            # Tabel transaksi
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS transactions (
                    id TEXT PRIMARY KEY,
                    product_id TEXT NOT NULL,
                    type TEXT NOT NULL,
                    quantity INTEGER NOT NULL,
                    timestamp TEXT NOT NULL,
                    notes TEXT,
                    FOREIGN KEY (product_id) REFERENCES products (id)
                )
            ''')

            # Index untuk optimasi query
            cursor.execute('CREATE INDEX IF NOT EXISTS idx_product_category ON products(category)')
            cursor.execute('CREATE INDEX IF NOT EXISTS idx_transaction_product ON transactions(product_id)')
            cursor.execute('CREATE INDEX IF NOT EXISTS idx_transaction_timestamp ON transactions(timestamp)')

            conn.commit()

    def _generate_id(self, prefix: str) -> str:
        """Generate unique ID dengan prefix dan timestamp"""
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
        unique_str = f"{prefix}_{timestamp}"
        return hashlib.md5(unique_str.encode()).hexdigest()

    def execute_query(self, query: str, params: tuple = (), fetch: bool = False):
        """Eksekusi query dengan optimasi connection pooling"""
        try:
            with sqlite3.connect(self.db_name) as conn:
                conn.row_factory = sqlite3.Row
                cursor = conn.cursor()
                cursor.execute(query, params)

                if fetch:
                    return cursor.fetchall()
                conn.commit()
        except sqlite3.Error as e:
            logging.error(f"Database error: {e}")
            raise

## Inventory Manager ##
class InventoryManager:
    def __init__(self):
        self.db = DatabaseManager()
        self.export_strategies = {
            'csv': CSVExportStrategy(),
            'json': JSONExportStrategy()
        }

    def add_product(self, name: str, category: str, price: float, quantity: int, min_stock: int = 5) -> Product:
        """Menambahkan produk baru ke inventori"""
        product_id = self.db._generate_id('prod')
        product = Product(product_id, name, category, price, quantity, min_stock)

        query = '''
            INSERT INTO products (id, name, category, price, quantity, min_stock)
            VALUES (?, ?, ?, ?, ?, ?)
        '''
        self.db.execute_query(query, (
            product.id, product.name, product.category,
            product.price, product.quantity, product.min_stock
        ))

        # Catat transaksi masuk awal
        self.record_transaction(product.id, 'in', quantity, 'Initial stock')

        logging.info(f"Added product: {product.name} (ID: {product.id})")
        return product

    def update_product(self, product_id: str, **kwargs) -> Optional[Product]:
        """Update informasi produk"""
        allowed_fields = {'name', 'category', 'price', 'quantity', 'min_stock'}
        updates = {k: v for k, v in kwargs.items() if k in allowed_fields}

        if not updates:
            return None

        set_clause = ", ".join(f"{k} = ?" for k in updates.keys())
        query = f"UPDATE products SET {set_clause} WHERE id = ?"

        self.db.execute_query(query, (*updates.values(), product_id))

        # Jika quantity diupdate, catat transaksi
        if 'quantity' in updates:
            old_quantity = self.get_product(product_id).quantity
            diff = updates['quantity'] - old_quantity
            if diff != 0:
                trans_type = 'in' if diff > 0 else 'out'
                self.record_transaction(product_id, trans_type, abs(diff), 'Quantity adjustment')

        logging.info(f"Updated product {product_id} with: {updates}")
        return self.get_product(product_id)

    def get_product(self, product_id: str) -> Optional[Product]:
        """Mendapatkan produk berdasarkan ID"""
        query = "SELECT * FROM products WHERE id = ?"
        result = self.db.execute_query(query, (product_id,), fetch=True)

        if not result:
            return None

        row = result[0]
        return Product(
            row['id'], row['name'], row['category'],
            row['price'], row['quantity'], row['min_stock']
        )

    def list_products(self, category: Optional[str] = None) -> List[Product]:
        """Mendapatkan daftar produk, bisa difilter berdasarkan kategori"""
        if category:
            query = "SELECT * FROM products WHERE category = ? ORDER BY name"
            results = self.db.execute_query(query, (category,), fetch=True)
        else:
            query = "SELECT * FROM products ORDER BY name"
            results = self.db.execute_query(query, fetch=True)

        return [
            Product(
                row['id'], row['name'], row['category'],
                row['price'], row['quantity'], row['min_stock']
            ) for row in results
        ]

    def record_transaction(self, product_id: str, trans_type: str, quantity: int, notes: str = "") -> Transaction:
        """Mencatat transaksi masuk/keluar"""
        if trans_type not in ('in', 'out'):
            raise ValueError("Transaction type must be 'in' or 'out'")

        transaction_id = self.db._generate_id('trans')
        timestamp = datetime.now().isoformat()
        transaction = Transaction(transaction_id, product_id, trans_type, quantity, timestamp, notes)

        query = '''
            INSERT INTO transactions (id, product_id, type, quantity, timestamp, notes)
            VALUES (?, ?, ?, ?, ?, ?)
        '''
        self.db.execute_query(query, (
            transaction.id, transaction.product_id, transaction.type,
            transaction.quantity, transaction.timestamp, transaction.notes
        ))

        # Update stok produk
        product = self.get_product(product_id)
        new_quantity = product.quantity + quantity if trans_type == 'in' else product.quantity - quantity
        self.update_product(product_id, quantity=new_quantity)

        logging.info(f"Recorded transaction: {trans_type} {quantity} of product {product_id}")
        return transaction

    def generate_report(self, report_type: str = 'stock', export_format: str = 'csv') -> str:
        """Membuat laporan dan mengeksport ke format yang diinginkan"""
        if report_type == 'stock':
            data = self._generate_stock_report()
            report_name = 'stock_report'
        elif report_type == 'transactions':
            data = self._generate_transaction_report()
            report_name = 'transaction_report'
        elif report_type == 'low_stock':
            data = self._generate_low_stock_report()
            report_name = 'low_stock_report'
        else:
            raise ValueError("Invalid report type")

        strategy = self.export_strategies.get(export_format)
        if not strategy:
            raise ValueError("Unsupported export format")

        filename = strategy.export(data)
        logging.info(f"Generated {report_type} report: {filename}")
        return filename

    def _generate_stock_report(self) -> List[Dict]:
        """Membuat laporan stok produk"""
        query = '''
            SELECT id, name, category, price, quantity, min_stock
            FROM products
            ORDER BY category, name
        '''
        results = self.db.execute_query(query, fetch=True)
        return [dict(row) for row in results]

    def _generate_transaction_report(self) -> List[Dict]:
        """Membuat laporan transaksi"""
        query = '''
            SELECT t.id, p.name as product_name, t.type, t.quantity, t.timestamp, t.notes
            FROM transactions t
            JOIN products p ON t.product_id = p.id
            ORDER BY t.timestamp DESC
            LIMIT 100
        '''
        results = self.db.execute_query(query, fetch=True)
        return [dict(row) for row in results]

    def _generate_low_stock_report(self) -> List[Dict]:
        """Membuat laporan produk dengan stok rendah"""
        query = '''
            SELECT id, name, category, price, quantity, min_stock
            FROM products
            WHERE quantity <= min_stock
            ORDER BY quantity ASC
        '''
        results = self.db.execute_query(query, fetch=True)
        return [dict(row) for row in results]

    def backup_data(self, backup_type: str = 'full') -> str:
        """Membuat backup data inventori"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"inventory_backup_{timestamp}.sql"

        try:
            with sqlite3.connect(self.db.db_name) as conn:
                with open(filename, 'w') as f:
                    for line in conn.iterdump():
                        f.write(f"{line}\n")

            logging.info(f"Created {backup_type} backup: {filename}")
            return filename
        except Exception as e:
            logging.error(f"Backup failed: {e}")
            raise

    def optimize_database(self):
        """Mengoptimasi performa database"""
        try:
            with sqlite3.connect(self.db.db_name) as conn:
                conn.execute("VACUUM")
                conn.execute("PRAGMA optimize")
                logging.info("Database optimization completed")
        except Exception as e:
            logging.error(f"Database optimization failed: {e}")
            raise

## Contoh Penggunaan ##
def main():
    # Inisialisasi sistem
    inventory = InventoryManager()

    # Tambahkan beberapa produk
    products = [
        ("Laptop", "Electronics", 1200.00, 10),
        ("Mouse", "Electronics", 25.50, 30),
        ("Notebook", "Stationery", 3.20, 50),
        ("Pen", "Stationery", 1.50, 100, 20)  # Contoh dengan min_stock custom
    ]

    for prod in products:
        inventory.add_product(*prod)

    # Update produk
    laptop = inventory.list_products(category="Electronics")[0]
    inventory.update_product(laptop.id, price=1150.00)

    # Catat transaksi
    inventory.record_transaction(laptop.id, 'out', 3, "Sold to customer")
    inventory.record_transaction(laptop.id, 'in', 5, "Restock from supplier")

    # Generate laporan
    inventory.generate_report('stock', 'csv')
    inventory.generate_report('low_stock', 'json')

    # Backup data
    inventory.backup_data()

    # Optimasi database
    inventory.optimize_database()

    print("Sistem inventori berjalan dengan baik. Lihat log untuk detail.")

if __name__ == "__main__":
    main()

Sistem inventori berjalan dengan baik. Lihat log untuk detail.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')